# Condenames Clue Generation using Word Embeddings

### Imports

In [1]:
import numpy as np
import random
import json
from tqdm.notebook import tqdm
from nltk.stem import PorterStemmer
from sentence_transformers import SentenceTransformer

### Loading word pool list(s)

In [2]:
with open('word_lists/codenames.txt') as f:
    original_words = np.array(f.read().splitlines())
with open('word_lists/duet.txt') as f:
    duet_words = np.array(f.read().splitlines())
with open('word_lists/deep_undercover.txt') as f:
    undercover_words = np.array(f.read().splitlines())
with open('clue_candidates.txt') as f:
    candidate_words = np.array(f.read().splitlines())

### Generate word grids
- Randomly samples words from word pool to fill word grid
- Assigns both player and model keys to words based on key card associations
    - Keys are either 'F'riendly, 'B'ystander, or 'A'ssassin
    - Derived from "Codenames: Duet" ruleset

In [3]:
WORD_POOL = duet_words
GRID_SIZE = 5

def generate_grids(word_pool=WORD_POOL):
    word_grid = np.random.choice(word_pool, GRID_SIZE**2, replace=False).reshape([GRID_SIZE,GRID_SIZE])
    key_cards = list(zip(
        ['A','B','B','B','B','B','F','F','F','F','F','F','F','F','F','B','A','B','B','B','B','B','B','B','A'], 
        ['F','F','F','F','F','F','F','F','F','B','B','B','B','B','A','A','A','B','B','B','B','B','B','B','B']
    ))
    random.shuffle(key_cards)
    key_grid_player, key_grid_model = zip(*key_cards)
    key_grid_player = np.reshape(key_grid_player, [GRID_SIZE,GRID_SIZE])
    key_grid_model = np.reshape(key_grid_model, [GRID_SIZE,GRID_SIZE])

    word_grid = [list(row) for row in word_grid]
    key_grid_player = [list(row) for row in key_grid_player]
    key_grid_model = [list(row) for row in key_grid_model]

    return word_grid, key_grid_player, key_grid_model

word_grid, key_grid_player, key_grid_model = generate_grids()

In [4]:
word_grid

[['BALLOON', 'ONION', 'SCRATCH', 'RADIO', 'QUACK'],
 ['MESS', 'CUCKOO', 'DOLLAR', 'DRYER', 'SHERLOCK'],
 ['BOWL', 'TATTOO', 'JOKER', 'MONA LISA', 'CLEOPATRA'],
 ['DRUM', 'AXE', 'PACIFIC', 'THUNDER', 'MONKEY'],
 ['BUBBLE', 'POWDER', 'CANE', 'WINDOW', 'HAMBURGER']]

In [5]:
key_grid_player

[['F', 'F', 'B', 'B', 'F'],
 ['B', 'F', 'B', 'B', 'B'],
 ['F', 'B', 'B', 'A', 'B'],
 ['A', 'F', 'A', 'B', 'B'],
 ['B', 'F', 'B', 'F', 'F']]

In [6]:
key_grid_model

[['F', 'A', 'F', 'A', 'F'],
 ['B', 'B', 'B', 'B', 'F'],
 ['F', 'B', 'F', 'F', 'F'],
 ['B', 'B', 'A', 'F', 'B'],
 ['B', 'B', 'B', 'B', 'B']]

### Get dictionary of word groups (for each key) based on word_grid and key_grid

In [7]:
def get_word_groups(word_grid, key_grid):
    word_groups = {}
    for i in range(len(key_grid)):
        for j in range(len(key_grid[i])):
            key = key_grid[i][j]
            word = word_grid[i][j]
            
            if key in word_groups:
                word_groups[key].append(word)
            else:
                word_groups[key] = [word]
    
    return word_groups

player_word_groups = get_word_groups(word_grid, key_grid_player)
model_word_groups = get_word_groups(word_grid, key_grid_player)

In [8]:
player_word_groups

{'F': ['BALLOON',
  'ONION',
  'QUACK',
  'CUCKOO',
  'BOWL',
  'AXE',
  'POWDER',
  'WINDOW',
  'HAMBURGER'],
 'B': ['SCRATCH',
  'RADIO',
  'MESS',
  'DOLLAR',
  'DRYER',
  'SHERLOCK',
  'TATTOO',
  'JOKER',
  'CLEOPATRA',
  'THUNDER',
  'MONKEY',
  'BUBBLE',
  'CANE'],
 'A': ['MONA LISA', 'DRUM', 'PACIFIC']}

In [9]:
model_word_groups

{'F': ['BALLOON',
  'ONION',
  'QUACK',
  'CUCKOO',
  'BOWL',
  'AXE',
  'POWDER',
  'WINDOW',
  'HAMBURGER'],
 'B': ['SCRATCH',
  'RADIO',
  'MESS',
  'DOLLAR',
  'DRYER',
  'SHERLOCK',
  'TATTOO',
  'JOKER',
  'CLEOPATRA',
  'THUNDER',
  'MONKEY',
  'BUBBLE',
  'CANE'],
 'A': ['MONA LISA', 'DRUM', 'PACIFIC']}

### Initialize word embedding model

In [10]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print(embedding_model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)


### Generate/load embeddings for candidate clue words

In [11]:
# candidate_embeddings = np.array([embedding_model.encode(word) for word in tqdm(candidate_words)])
# np.save("clue_candidate_embeddings.npy", candidate_embeddings)
    
candidate_embeddings = {
    word:embedding for word, embedding in zip(candidate_words, np.load("clue_candidate_embeddings.npy"))
}

### Function to filter candidate clue words based on grid words
- Words on the grid should not be allowed as clues
- Matching includes substring, superstrings, and stemmed strings

In [12]:
def filter_dict_by_stemmed_keys(data_dict, word_list):
    stemmer = PorterStemmer()

    # Stem all words in the list
    stemmed_words = {stemmer.stem(word) for word in word_list}

    def should_remove(key):
        """Check if a given key should be removed based on the conditions."""
        stemmed_key = stemmer.stem(key)
        for word in stemmed_words:
            if stemmed_key == word or word in stemmed_key or stemmed_key in word:
                return True
        return False

    # Filter dictionary based on the should_remove condition
    filtered_dict = {key: value for key, value in data_dict.items() if not should_remove(key)}

    return filtered_dict

### Util functions for clue generation

In [13]:
def normalize(v):
    """Return the unit vector of v."""
    norm = np.linalg.norm(v)
    return v if norm == 0 else v / norm

def cosine_similarity(a, b):
    """Compute cosine similarity between two vectors."""
    return np.dot(a, b)

def nearest_neighbor(v_star, candidate_embs):
    """
    Given an ideal clue vector v_star and a dictionary mapping candidate clue words
    to their precomputed embeddings, return the candidate with the highest cosine similarity.
    """
    best_word = None
    best_sim = -np.inf
    for word, emb in candidate_embs.items():
        sim = cosine_similarity(v_star, emb)
        if sim > best_sim:
            best_sim = sim
            best_word = word
    return best_word

### Function computes and optimal clue word and clue count based on game state parameters
- See docstring

In [14]:
def find_best_cluster(
    friend_words, 
    non_friend_words,
    friend_embs,  
    non_friend_embs, 
    candidate_embs,
    target_N=3, 
    penalty_coeff=0.2,
    verbose=True
):
    """
    For each candidate cluster size N (from 1 to len(friend_words)), this function selects
    a tightly grouped cluster of friendly words (using a greedy strategy) and computes its centroid.
    It then evaluates a margin — defined as:
    
         margin = min_{f in cluster} cosine_similarity(centroid, f)
                  - max_{w in non-friends} cosine_similarity(centroid, w)
    
    To encourage clusters in a desired range (e.g. 2 to 4), we subtract a quadratic penalty:
    
         penalty = penalty_coeff * (N - target_N)^2
    
    The overall score is then:
    
         score = margin - penalty
    
    The best cluster (and corresponding centroid) is chosen based on the highest overall score.
    
    Parameters:
      - friend_words: list of friendly words.
      - non_friend_words: list of non-friendly words (e.g., bystanders + assassins).
      - candidate_words: list of candidate clue words.
      - friend_embs: dict mapping each friendly word to its embedding.
      - non_friend_embs: dict mapping each non-friendly word to its embedding.
      - candidate_embs: dict mapping each candidate clue word to its embedding.
      - target_N: desired number of friendlies (default 3).
      - penalty_coeff: coefficient for the quadratic penalty (default 0.2).
    
    Returns:
      - best_cluster: list of friendly words in the chosen cluster.
      - best_N: the size of the cluster.
      - best_v_star: the centroid vector of the best cluster.
      - best_clue: the candidate clue word (from candidate_words) whose embedding is closest to best_v_star.
    """
    best_score = -np.inf
    best_cluster = None
    best_N = None
    best_v_star = None

    # Loop over all possible cluster sizes.
    for N in range(1, len(friend_words) + 1):
        if verbose:
            print(f"N = {N}")
            print()
        # Try every friendly word as a potential seed for the cluster.
        for seed in friend_words:
            if verbose:
                print(f"Cluster seed = \"{seed}\"")
            cluster = [seed]
            remaining = set(friend_words) - {seed}
            # Greedily grow the cluster until it has N words.
            while len(cluster) < N and remaining:
                best_candidate = None
                best_avg_sim = -np.inf
                for candidate in remaining:
                    # Compute average cosine similarity between candidate and the current cluster.
                    sim_sum = sum(cosine_similarity(friend_embs[candidate], friend_embs[w])
                                  for w in cluster)
                    avg_sim = sim_sum / len(cluster)
                    if avg_sim > best_avg_sim:
                        best_avg_sim = avg_sim
                        best_candidate = candidate
                if best_candidate is not None:
                    cluster.append(best_candidate)
                    remaining.remove(best_candidate)
                else:
                    break
            # Only consider complete clusters of size N.
            if len(cluster) != N:
                continue
            # Compute the centroid of the cluster.
            centroid = normalize(sum(friend_embs[w] for w in cluster))
            # Calculate the minimum similarity from the cluster words to the centroid.
            friend_sims = [cosine_similarity(centroid, friend_embs[w]) for w in cluster]
            min_friend_sim = min(friend_sims)
            # Calculate the maximum similarity from any non-friendly word to the centroid.
            non_friend_sims = [cosine_similarity(centroid, non_friend_embs[w]) for w in non_friend_words]
            max_non_friend_sim = max(non_friend_sims) if non_friend_sims else -np.inf
            # Define a margin that measures the separation.
            margin = min_friend_sim - max_non_friend_sim
            # Optionally, you could add a bonus for larger clusters (e.g., margin + alpha * N).
            score = margin - (penalty_coeff * (N - target_N)**2)
            if verbose:
                print(f"Cluster words = {cluster}")
                print(f"Cluster margin = {margin}")
                print(f"Cluster score = {score}")
                print(f"Cluster clue = {nearest_neighbor(centroid, candidate_embs)}")
                print()
            if score > best_score:
                best_score = score
                best_cluster = list(cluster)
                best_N = N
                best_v_star = centroid
        if verbose:
            print()

    # Once the best centroid is chosen, "round" it to a candidate clue word.
    best_clue = nearest_neighbor(best_v_star, candidate_embs)
    if verbose:
        print(f"BEST N: {best_N}")
        print(f"BEST CLUSTER: {best_cluster}")
        print(f"BEST SCORE: {best_score}")
        print(f"BEST CLUE: {best_clue}")
    return best_cluster, best_N, best_v_star, best_clue

## Putting it all together

In [16]:
# Re-generate random word and key grids
word_grid, key_grid_player, key_grid_model = generate_grids()
player_word_groups = get_word_groups(word_grid, key_grid_player)
model_word_groups = get_word_groups(word_grid, key_grid_player)

# Separate the friendly words and non-friendlies (bystanders and assassins)
friend_words = player_word_groups['F']
non_friend_words = player_word_groups['B'] + player_word_groups['A']
all_words = friend_words + non_friend_words

# Filter clue candidate embeddings based on grid words
candidate_embeddings = filter_dict_by_stemmed_keys(candidate_embeddings, all_words)

# Compute embeddings for grid words
friend_embeddings = {word: embedding_model.encode(word) for word in friend_words}
non_friend_embeddings = {word: embedding_model.encode(word) for word in non_friend_words}

print(json.dumps(player_word_groups, indent=4))
print()
print("Friend Words:", friend_words)
print("Non-Friend Words:", non_friend_words)
print()
print(f"Num candidate clues = {len(candidate_embeddings)}")
print(f"Num candidate clues (filtered) = {len(candidate_embeddings)}")
print()

best_cluster, best_N, best_v_star, best_clue = \
    find_best_cluster(
        friend_words, 
        non_friend_words, 
        friend_embeddings, 
        non_friend_embeddings, 
        candidate_embeddings,
        target_N=3, 
        penalty_coeff=0.2,
        verbose=True
    )

{
    "B": [
        "MISS",
        "SAHARA",
        "FARM",
        "HOMER",
        "HELMET",
        "ANCHOR",
        "WHISTLE",
        "BIG BEN",
        "LUNCH",
        "PEN",
        "BATTLE",
        "LOCUST",
        "ACE"
    ],
    "F": [
        "JUDGE",
        "MONKEY",
        "IGLOO",
        "EINSTEIN",
        "SALAD",
        "PENNY",
        "FIDDLE",
        "HAWAII",
        "SPRAY"
    ],
    "A": [
        "BRASS",
        "MEMORY",
        "TRICK"
    ]
}

Friend Words: ['JUDGE', 'MONKEY', 'IGLOO', 'EINSTEIN', 'SALAD', 'PENNY', 'FIDDLE', 'HAWAII', 'SPRAY']
Non-Friend Words: ['MISS', 'SAHARA', 'FARM', 'HOMER', 'HELMET', 'ANCHOR', 'WHISTLE', 'BIG BEN', 'LUNCH', 'PEN', 'BATTLE', 'LOCUST', 'ACE', 'BRASS', 'MEMORY', 'TRICK']

Num candidate clues = 67758
Num candidate clues (filtered) = 67758

N = 1

Cluster seed = "JUDGE"
Cluster words = ['JUDGE']
Cluster margin = 0.6633400321006775
Cluster score = -0.13665996789932255
Cluster clue = JUDICIAL

Cluster seed = "MO